# LCEL 체인을 통한 챗봇 구성

## OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# .env 파일 로드
load_dotenv()

print(f"{os.getenv("OPENAI_API_KEY")[:9]}***")

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print(llm.model_name)


## 프롬프트 메시지 구성
역할 기반 프롬프트 템플릿 정의
- system 역할: LLM에게 '명탐정 코난 전문가' 역할 부여
- human 역할: 사용자 질문 삽입

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 '명탐정 코난' 전문가입니다. 모든 질문에 친절하고 정확하게 답해주세요."),
    ("human", "{question}")
])

## 체인 구성 및 UI 입출력 함수 정의

### LCEL 체인 구성 

체인 (프롬프트 → LLM → 출력 파서)를 파이프(|)로 연결
- `ChatPromptTemplate`: 문자열 메시지 생성
- `LLM`: 응답 생성
- `StrOutputParser`: LLM 응답에서 텍스트 추출

### Gradio 입출력 함수 정의
- `conan_expert_reply` : 사용자 입력값을 받아 LCEL 체인에 전달하고 결과 리턴


In [ ]:
# LCEL 체인 구성
chain = prompt | llm | StrOutputParser()

# Gradio 입출력 함수 정의
def conan_expert_reply(user_input):
    return chain.invoke({"question": user_input})

## Gradio UI 구성
### Gradio에서 자주 사용하는 UI 컴포넌트 유형
| 분류       | 컴포넌트            | 설명                           | 입력/출력  |
| -------- | --------------- | ---------------------------- | ------ |
| 텍스트      | `Textbox`       | 단일 또는 여러 줄 텍스트 입력            | 입력/출력  |
|          | `Label`         | 텍스트 라벨 출력 (주로 출력 용도)         | 출력     |
|          | `Markdown`      | 마크다운 형식 텍스트 출력               | 출력     |
| 버튼 및 컨트롤 | `Button`        | 클릭 가능한 버튼                    | 입력 트리거 |
|          | `Radio`         | 여러 선택지 중 하나 선택 (라디오 버튼)      | 입력     |
|          | `Checkbox`      | 단일 체크박스 (참/거짓 선택)            | 입력     |
|          | `CheckboxGroup` | 여러 체크박스 중 다중 선택              | 입력     |
|          | `Dropdown`      | 드롭다운 메뉴에서 선택                 | 입력     |
|          | `Slider`        | 숫자 범위를 슬라이더로 선택              | 입력     |
|          | `Number`        | 숫자 직접 입력                     | 입력     |
| 멀티미디어    | `Image`         | 이미지 입력 또는 출력                 | 입력/출력  |
|          | `Audio`         | 오디오 입력 또는 출력                 | 입력/출력  |
|          | `Video`         | 비디오 입력 또는 출력                 | 입력/출력  |
|          | `File`          | 파일 업로드 및 다운로드                | 입력/출력  |
| 복합 구성    | `Chatbot`       | 대화형 채팅 UI, 메시지 목록 표시 및 입력 처리 | 입력/출력  |
|          | `State`         | 상태 저장용 숨겨진 변수                | 내부 처리  |
|          | `Dataframe`     | 판다스 DataFrame 표 형태로 입력 또는 출력 | 입력/출력  |
|          | `JSON`          | JSON 형식 데이터 입력 또는 출력         | 입력/출력  |
| 레이아웃     | `Row`, `Column` | UI 구성 요소들을 가로/세로로 정렬하는 컨테이너  | 레이아웃   |
|          | `Tabs`, `Tab`   | 탭 메뉴를 구성                     | 레이아웃   |
|          | `Accordion`     | 접이식 섹션                       | 레이아웃   |
| 웹뷰어      | `HTML`          | HTML 형식 출력                   | 출력     |
|          | `Code`          | 코드 블럭 형식 출력                  | 출력     |
|          | `Gallery`       | 이미지들을 갤러리 형태로 출력             | 출력     |


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가에게 질문해보세요!")
    question = gr.Textbox(label="질문을 입력하세요")
    answer = gr.Textbox(label="코난 전문가의 답변", lines=5)
    question.submit(fn=conan_expert_reply, inputs=question, outputs=answer)

# Gradio 실행
demo.launch()

-----
** End of Documents **